<a href="https://colab.research.google.com/github/ggzlemos/ArvoreDecisao/blob/main/ArvoreDecisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Este notebook implementa o algoritmo de árvore de decisão. Após a implementação, é feito um teste utilizando o wine dataset, disponibilizado pela biblioteca sklearn.

##Esta atividade faz parte do treinamento básico do Hub de Inovação em Intligência Artifical (H2IA)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

###Nesta implementação, utiliza-se o método da entropia para calcular o ganho de informação dos nodos da árvore

In [ ]:
def entropia(y):
  occurences = np.bincount(y)
  frac = occurences / len(y)
  lista = []
  for i in frac:
    if i != 0:
      lista.append(i *np.log2(i))
  return -sum(lista)
  

###Classe que modela cada nodo da árvore

In [ ]:
class Nodo:

  def __init__(self):
    self.esquerda = None
    self.direita = None
    self.classe = None
    self.atributo = None
    self.threshold = None
 
  def decide_classe(self, y): 
    counts = np.bincount(y)
    self.classe = np.argmax(counts)

  def set_esquerda(self, nodo):
    self.esquerda = nodo

  def set_direita(self, nodo):
    self.direita = nodo 

  def get_classe(self):
    return self.classe  


###Classe da árvore de decisão

In [ ]:
class Arvore:

  def __init__(self, max_depth=100, min_sample_split=2):
    self.raiz = None
    self.max_depth = max_depth
    self.min_sample_split = min_sample_split

  def fit(self, X, y):
    self.raiz = self.grow_tree(X, y)    

  def grow_tree(self, X, y, depth=0):
    num_samples, atributos = X.shape
    if depth > self.max_depth or len(np.unique(y)) == 1 or num_samples < self.min_sample_split:
      nodo = Nodo()
      nodo.decide_classe(y)
      return nodo
 
    attr, threshold = self.get_attr_thres(X, y)


    #após, dividir em esquerda (menor ou igual) e direita
    esq, dir = self.split(X[:, attr], threshold)
    nodo_esq = self.grow_tree(X[esq, :], y[esq], depth+1)
    nodo_dir = self.grow_tree(X[dir, :], y[dir], depth+1)

    nodo = Nodo()
    nodo.threshold = threshold
    nodo.atributo = attr
    nodo.set_esquerda(nodo_esq)
    nodo.set_direita(nodo_dir)


    return nodo

  def get_attr_thres(self, X, y):
    columns = X.shape[1]

    #busca gulosa pelo melhor par (atributo, threshold)
    best_gain = -1
    best_attr = None
    best_threshold = None


    for atributo in range(columns):
      coluna = X[:, atributo]     
      coluna_unicos = np.unique(X[:, atributo])
      for threshold in coluna_unicos:
        esq, dir = self.split(coluna, threshold)

        if len(esq) == 0 or len(dir) == 0:
          gain = 0
        else:  
          gain = self.info_gain(y, esq, dir)

        if gain > best_gain:
          best_gain = gain
          best_attr = atributo
          best_threshold = threshold
  
    return best_attr, best_threshold

  def info_gain(self, y, esquerda, direita):
    entropia_pai = entropia(y)

    entropia_esq = entropia(y[esquerda])
    entropia_dir = entropia(y[direita])

    somador = ((len(y[esquerda]) / len(y)) * entropia_esq + 
               (len(y[direita]) / len(y)) * entropia_dir)

    ig = entropia_pai - somador

    return ig

  def split(self, X, threshold):

    esq = np.argwhere(X <= threshold).flatten()
    dir = np.argwhere(X > threshold).flatten()

    return esq, dir

  def predict(self, X):
      results = []
      for e in X:
        classe = self.go_down_tree(e, self.raiz)
        results.append(classe)   
      return np.array(results)

  def go_down_tree(self, x, nodo):
 
      if nodo.esquerda == None and nodo.direita == None:      
        a = nodo.get_classe()         
        return a
      
      if x[nodo.atributo] <= nodo.threshold:
        return self.go_down_tree(x, nodo.esquerda)
      else:
        return self.go_down_tree(x, nodo.direita)
  

###Teste com o dataset de vinhos

In [ ]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target)

clf_1 = Arvore()
clf_1.fit(X_train, y_train)
result = clf_1.predict(X_test)

###Cálculo simplesda acurácia do modelo

In [ ]:
acuracia = np.sum(result == y_test) / len(y_test)
acuracia

0.9111111111111111

###Referências utilizadas para o desenvolvimento deste notebook:

[Aprendizado com árvores de decsião](https://ricardomatsumura.medium.com/aprendizado-com-%C3%A1rvores-de-decis%C3%A3o-73d874664d1)

[Árvores de Decisão](http://web.tecnico.ulisboa.pt/ana.freitas/bioinformatics.ath.cx/bioinformatics.ath.cx/indexf23d.html?id)

[ID3 Algorithm (Wiki)](https://en.wikipedia.org/wiki/ID3_algorithm)

[Árvores de Decisão](https://edisciplinas.usp.br/pluginfile.php/4469825/mod_resource/content/1/ArvoresDecisao_normalsize.pdf)

[StatsQuest - Decision Trees](https://www.youtube.com/watch?v=7VeUPuFGJHk)

[Machine Learning from Scratch](https://www.youtube.com/watch?v=Bqi7EFFvNOg)
